# ML/DL 管理工具

本文主要记录Pytorch下常用的deep learning管理工具。主要参考资料：

- [ML/DL Engineering Made Easy with PyTorch’s Ecosystem Tools](https://medium.com/pytorch/ml-dl-engineering-made-easy-with-pytorchs-ecosystem-tools-13e4242bfb54)
- [Accelerate your Hyperparameter Optimization with PyTorch’s Ecosystem Tools](https://medium.com/pytorch/accelerate-your-hyperparameter-optimization-with-pytorchs-ecosystem-tools-bc17001b9a49)

## 工程化ML/DL

在开始超参数优化前，先了解如何利用PyTorch的生态系统工具轻松启动ML / DL项目。

Allegro Trains是Allegro AI开发的开源机器学习和深度学习实验管理器以及ML-Ops解决方案，可提高GPU利用率以及AI团队的效率和生产力。该解决方案提供了功能强大，可扩展的实验管理和ML-Ops工具。作为PyTorch生态系统的一部分，Allegro Trains帮助PyTorch的研究人员和开发人员更轻松地管理复杂的机器学习项目。Allegro Trains与数据无关，可以应用于结构化数据，图像处理，音频处理，文本处理等。

### 深度学习项目中的挑战

每个数据科学家都知道，机器/深度学习项目所包含的任务远远不止选择模型架构和对数据进行模型训练。机器学习项目包括不断进行的试验和试错，这些尝试与实验室科学实验相似，而不仅仅是软件开发工作流程。这就是训练通常被称为实验，且需要平台来帮助管理的原因。

这种独特的工作流程带来了一些挑战，这些挑战可能会使机器学习项目收敛到所需结果的时间大大延长。

由于项目包含大量实验，因此需要一种简单的方法来跟踪这些实验，记录所有正在使用的超参数值，并能够在新数据到达数周和数月后重现成功的实验。为此，您需要将实验管理工具集成到工作流程中。

另一个已知的挑战是DevOps的工作，如果管理不当，可能会消耗大量数据科学家的时间。训练模型是一个耗时的过程-甚至数小时甚至数天。因此，通常的做法是分配训练机器（本地和云机器）来执行，而数据科学家自己的机器则保留用于项目代码库的进一步开发。要管理所有这些机器，需要持续的DevOps努力：设置机器，引入队列管理系统以管理发送到机器的实验流并监视机器的状态（GPU，CPU，内存）。

此外，随着时间的流逝，不同的数据科学家将使用不同版本的深度学习框架，例如PyTorch，它们需要不同版本的Cuda，Cudnn和其他软件包，还可能使用不同的Python软件包版本。对不同实验为每台机器创建并扩展足够的容器可能会很麻烦。

为了解决这些挑战以及更多挑战，建议在PyTorch生态系统中使用Allegro Trains和其他工具。
让我们以一个简单的图像处理示例为例，演示这些工具如何帮助解决这些挑战。

### 使用 Allegro Trains 应对挑战

我们将讨论的第一个工具是Allegro Trains实验管理器和ML-Ops解决方案。Allegro Trains支持实验跟踪，分析，比较，超参数调整，自动化，可重复性和各种其他功能。它是由三个开源组件组成的套件：

- [Trains Python程序包](https://github.com/allegroai/clearml)：此Python程序包允许您仅两行代码即可将Allegro Trains集成到您的Python代码中。
- [Trains服务器](https://github.com/allegroai/clearml-server)-这是Allegro Trains的后端基础结构。可以采用多种格式部署自己的自托管Trains Server，包括用于Linux，Windows 10，macOS，预构建的AWS EC2 AMI和Kubernetes的预构建Docker映像。
- [Trains Agent](https://github.com/allegroai/clearml-agent)—Trains Agent 是Allegro Trains 的DevOps组件，可实现远程实验执行，资源控制和自动化（例如，Trains内置贝叶斯超参数优化）。

Allegro Trains的一大优点是与PyTorch的集成工作为零。您只需要在主脚本的顶部添加以下两行代码，就可以开始了：

```Python
from trains import Task
task = Task.init(project_name='Image Classification', task_name='image_classification_CIFAR10')
```

Allegro Trains将您的PyTorch开发组织到项目中，因此一旦执行代码，您的实验就会记录在Web应用程序中的相关项目下。现在，我们将代码集成到了Allegro Trains上，我们可以享受它附带的所有好处：

- 重现性—每个实验的所有执行数据都自动（自动）记录在Trains Server上：git存储库，分支，提交ID，未提交的更改以及所有使用的Python包（包括执行时的特定版本）。这样可以确保随时重现实验。我们都熟悉软件包版本更改且脚本不再起作用的情况。此功能有助于我们避免对此类令人沮丧的情况进行故障排除。
- 改进的团队合作精神-借助Allegro Trains，可以持续共享每个团队成员的工作，从而可以在视觉上集体讨论结果，与团队成员有效地讨论问题和可能的补救措施，并共享在一个用例上表现良好的实验，可以轻松地应用于另一个案例，在另一个项目上工作的队友的用例。
- 轻松进行实验跟踪和分析-Allegro Trains Web应用程序包括各种分析和比较工具，例如创建排行榜，根据所选指标，平行坐标等对团队的所有实验进行排名。

使用Allegro Trains还有许多其他功能并从中受益，稍后讨论其中一些功能。现在，回到代码。

接下来，要确保所有参数都反映在实验管理器Web应用程序中，并且代码中没有隐藏“魔术数字”。您可以使用众所周知的argparse包裹，Allegro Trains会自动将其捡起。或者，您可以只定义配置字典并将其连接到Allegro Trains任务对象：

```Python
configuration_dict = {'number_of_epochs': 3, 'batch_size': 4, 'dropout': 0.25, 'base_lr': 0.001}
configuration_dict = task.connect(configuration_dict)
```

现在是时候定义我们的PyTorch Dataset对象了。使用PyTorch的域库随附的内置数据集--CIFAR10数据集，该数据集可以通过Torchvision轻松加载。还将使用PyTorch的DataLoader，可在数据集上迭代。DataLoader支持单进程或多进程加载，自定义加载顺序以及可选的自动批处理（归类）和内存固定：

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import torchvision.datasets as datasets
import torchvision.transforms as transforms

from clearml import Task

In [3]:
task = Task.init(project_name='Image Example', task_name='image classification CIFAR10')
configuration_dict = {'number_of_epochs': 3, 'batch_size': 4, 'dropout': 0.25, 'base_lr': 0.001}
configuration_dict = task.connect(configuration_dict)  # enabling configuration override by clearml
print(configuration_dict)  # printing actual configuration (after override in remote mode)

ClearML Task: created new task id=bafc9a78dcdc44c18984b5503d923951
======> WARNING! Git diff to large to store (751kb), skipping uncommitted changes <======
ClearML results page: https://demoapp.demo.clear.ml/projects/9cb82c9240194bb99646a1977090987d/experiments/bafc9a78dcdc44c18984b5503d923951/output/log
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
{'number_of_epochs': 3, 'batch_size': 4, 'dropout': 0.25, 'base_lr': 0.001}


In [4]:
transform = transforms.Compose([transforms.ToTensor()])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=configuration_dict.get('batch_size', 4),
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=configuration_dict.get('batch_size', 4),
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

device = torch.cuda.current_device() if torch.cuda.is_available() else torch.device('cpu')

Files already downloaded and verified
Files already downloaded and verified


C:\Users\11445\miniconda3\envs\hydrus\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


接下来，定义模型。PyTorch实现了轻松灵活的模型配置，从而使研究模型体系结构变得尽可能容易。示例创建一个简单的模型：

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.dorpout = nn.Dropout(p=configuration_dict.get('dropout', 0.25))
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(self.dorpout(x))
        return x

In [6]:
net = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=configuration_dict.get('base_lr', 0.001), momentum=0.9)

注意，torchvision还包括一组有用的流行的预训练模型，这些模型可以作为要构建的每个模型的重要基础。

尽管TensorBoard是TensorFlow框架的一部分，但它也已完全集成到PyTorch中。这样便可以轻松地报告标量值（例如，损耗，准确性等），图像（例如，出于调试目的而馈入模型的图像）等等。这是Allegro Trains的另一个自动功能，它将自动提取发送到TensorBoard的所有报告，并将它们记录在您的实验下的Web应用程序中。因此，让我们将TensorBoard添加到我们的代码中，并在训练期间使用它来报告损失值：

In [7]:
tensorboard_writer = SummaryWriter('./tensorboard_logs')

在每个epoch结束时的测试期间，我们可以报告一些调试图像，这些图像带有其地面真实标签和模型做出的预测，以及每个标签的准确性和总准确性：

In [8]:
def test_model(test_dataloader, iteration):
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for j, data in enumerate(test_dataloader, 1):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(len(images)):
                label = labels[i].item()
                class_correct[label] += c[i].item()
                class_total[label] += 1
            
            if j % 500 == 0:    # report debug image every 500 mini-batches
                for n, (img, pred, label) in enumerate(zip(images, predicted, labels)):
                    tensorboard_writer.add_image("testing/{}-{}_GT_{}_pred_{}"
                                                 .format(j, n, classes[label], classes[pred]), img, iteration)

    for i in range(len(classes)):
        class_accuracy = 100 * class_correct[i] / class_total[i]
        print('[Iteration {}] Accuracy of {} : {}%'.format(iteration, classes[i], class_accuracy))
        tensorboard_writer.add_scalar('accuracy per class/{}'.format(classes[i]), class_accuracy, iteration)

    total_accuracy = 100 * sum(class_correct)/sum(class_total)
    print('[Iteration {}] Accuracy on the {} test images: {}%\n'.format(iteration, sum(class_total), total_accuracy))
    tensorboard_writer.add_scalar('accuracy/total', total_accuracy, iteration)

In [9]:
for epoch in range(configuration_dict.get('number_of_epochs', 3)):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 1):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
        iteration = epoch * len(trainloader) + i
        if i % 2000 == 0:    # report loss every 2000 mini-batches
            print('[Epoch %d, Iteration %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 2000))
            tensorboard_writer.add_scalar('training loss', running_loss / 2000, iteration)
            running_loss = 0.0
    
    test_model(testloader, iteration)

print('Finished Training')

[Epoch 1, Iteration  2001] loss: 2.298
[Epoch 1, Iteration  4001] loss: 2.136
[Epoch 1, Iteration  6001] loss: 1.903
[Epoch 1, Iteration  8001] loss: 1.751
[Epoch 1, Iteration 10001] loss: 1.687
[Epoch 1, Iteration 12001] loss: 1.630
[Iteration 12500] Accuracy of plane : 54.6%
[Iteration 12500] Accuracy of car : 66.4%
[Iteration 12500] Accuracy of bird : 16.7%
[Iteration 12500] Accuracy of cat : 13.0%
[Iteration 12500] Accuracy of deer : 46.0%
[Iteration 12500] Accuracy of dog : 42.9%
[Iteration 12500] Accuracy of frog : 50.3%
[Iteration 12500] Accuracy of horse : 43.8%
[Iteration 12500] Accuracy of ship : 53.8%
[Iteration 12500] Accuracy of truck : 47.6%
[Iteration 12500] Accuracy on the 10000.0 test images: 43.51%

[Epoch 2, Iteration  2001] loss: 1.550
[Epoch 2, Iteration  4001] loss: 1.520
[Epoch 2, Iteration  6001] loss: 1.479
[Epoch 2, Iteration  8001] loss: 1.463
[Epoch 2, Iteration 10001] loss: 1.434
[Epoch 2, Iteration 12001] loss: 1.426
[Iteration 25000] Accuracy of plane : 4

最后，在实验结束时，要保存模型：

In [10]:
PATH = './cifar_net4clearml.pth'
torch.save(net.state_dict(), PATH)
tensorboard_writer.close()

In [11]:
print('Task ID number is: {}'.format(task.id))

Task ID number is: bafc9a78dcdc44c18984b5503d923951


Allegro Trains将自动检测到正在保存模型，并将模型位置注册到Allegro服务器中的experiment 中。建议将模型的副本上载到集中存储（本地或云），如果有可用的存储，Allegro Trains可以为您完成。初始化Allegro Trains时只需轻松地添加的集中存储位置的任务目标，快照将被存储在指定的存储，并且将被链接到您的实验，这样就能从任何机器访问它。

现在，我们要做的就是运行实验并在webapp中观察其进度。Allegro Trains不仅会记录您的所有报告，而且还会在机器及其GPU上添加状态报告。

注意：由于我们没有安装Trains-server，因此该实验将记录在Allegro Trains演示服务器上。该演示服务器旨在轻松快速地试验Allegro Trains套件。如果未安装自托管的Trains服务器，则Allegro Trains Python软件包将自动与演示服务器一起使用。

要查看这些报告，直接在浏览器中输入前面 “ClearML results page:”后显示的url即可。

现在出现了Allegro Trains的最佳功能之一：一旦将实验记录到Trains Server中，您就可以享受ML-Ops功能。您需要做的就是在指定用于执行实验的每台机器上安装Trains Agent。Trains Agent是一个守护程序，可以为您旋转一个容器并运行您的代码。无论是办公计算机还是云计算机，Trains Agent都将为团队成员提供完整的队列管理和机器诊断功能。

trains-agent的安装非常简单：

```Shell
pip install trains-agent
```

安装后，您将执行一个简单的配置命令并按照说明进行操作：

```Shell
trains-agent init
```

为了将机器注册到Trains Server中，您只需在webapp中创建队列并将机器设置为侦听特定队列：

```Shell
trains-agent daemon -queue my_queue
```

您所要做的就是在Web应用程序中选择任务，更改超级参数，然后Trains-Agent将负责其余的工作。在运行实验时，您还可以从DockerHub中命名一个容器，以供Trains Agent使用。例如，声明requested docker为“nvidia/cuda:10.0-cudnn7-runtime”，您的实验将在此docker内执行。现在，每当您在容器中需要新的Cuda版本时，就可以停止骚扰DevOps团队。还可以从Web应用程序更新Python软件包的版本，因此，每次要更新numpy软件包的版本时，都不需要新的容器。

机器学习是一个令人兴奋的领域，具有许多应用程序。但是，管理机器学习项目包括解决许多独特的挑战。公司倾向于忽略花费时间和精力来构建有助于应对这些挑战的基础架构，或者尝试自己构建它。在这两种情况下，他们最终都将精力转移到了机器和深度学习的核心研发上。PyTorch生态系统包括一组开源工具，这些工具一旦集成到您的工作流程中，将提高您的机器学习团队的生产力。
在本教程中，我们演示了如何使用对TensorBoard的Allegro Trains，Torchvision和PyTorch内置支持来实现更简单和高效的机器和深度学习工作流程。零集成工作量和零成本，您将获得一个实验管理系统和一个ML-Ops解决方案。

要了解有关Allegro Trains的更多信息，请参考其[文档](https://allegro.ai/docs/?utm_source=pytorch_blog&utm_medium=referral&utm_campaign=trains_c)。在本系列的下一篇博客文章中，我们将演示如何利用此处讨论的工具在实验中启动自动贝叶斯优化超参数搜索。